In [1]:
version = 'v20250521'

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import copy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold,cross_val_predict
from skmap.misc import find_files, GoogleSheet, ttprint
import joblib
import numpy as np

# discretinized points
df = pd.read_parquet('./material/agg_pixel_change.pq')
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
df = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:3035")
print(df.shape)

# # samples # in the end, we didn't opt for sample vs. pixel comparison
# spl = pd.read_parquet(f'./material/srs_agg_{version}.pq')
# geometry = [Point(xy) for xy in zip(spl['lon_mean'], spl['lat_mean'])]
# spl = gpd.GeoDataFrame(spl, geometry=geometry, crs="EPSG:4032")
# spl = spl.to_crs('EPSG:3035')

sizes = [2.5, 10, 40, 200]

(1167788, 15)


In [2]:
for isize in sizes:
    print(isize, '-------------------------------')
    plg = gpd.read_file(f'./material/grid_all.{isize}km_agg.{version}.gpkg')
    df = gpd.sjoin(df, plg[['geometry']], how="inner", predicate="intersects")
    df = df.rename(columns={'index_right':f'id_{isize}'})
    
    # # randomly select 2500 per id_isize as 1, others 0
    # idx = (df.groupby(f'id_{isize}', group_keys=False)
    #          .apply(lambda g: g.sample(n=min(2500, len(g)), random_state=42))
    #          .index)
    # df[f'is_{isize}'] = df.index.isin(idx).astype(int)


2.5 -------------------------------
10 -------------------------------
40 -------------------------------
200 -------------------------------


## spatial aggregation

In [13]:
# point
from scipy.spatial import distance_matrix

# correlation function
def exponential_model(h, nugget, sill, range_):
    return nugget + (sill - nugget) * (1 - np.exp(-h / range_))

def correlation_function(h, nugget, sill, range_):
    # Calculate the variogram value at distance h
    gamma_h = exponential_model(h, nugget, sill, range_)
    # Calculate and return the correlation function value
    return (sill - gamma_h) / sill

params_es = np.array([0.74285565, 1.00000001, 5.99345285])
params_de = np.array([0.72415892,  1.00000001, 10.60109839])

# get the map based spatial aggregates
for isize in sizes:
    ttprint(isize, '-------------------------------')
    plg = gpd.read_file(f'./material/grid_all.{isize}km_agg.{version}.gpkg')
    plg_id_col = f'id_{isize}'
    group_ids = df[plg_id_col].dropna().unique().tolist()

    for id_val in group_ids:
        ipnt = df.loc[df[plg_id_col] == id_val]
        if ipnt.empty:
            continue
        
        params_vg = params_es if ipnt.iloc[0]['nuts0'] == 'ES' else params_de
 
        # distance
        coords = np.array([(geom.x, geom.y) for geom in ipnt.geometry])
        dist_matrix = distance_matrix(coords, coords) / 1000
        corr_matrix = correlation_function(dist_matrix, *params_vg)
        np.fill_diagonal(corr_matrix, 1.0)
        
        # variances
        std09 = ipnt['pred_std_2009'].to_numpy()
        var_matrix_2009 = np.outer(std09, std09) * corr_matrix
        std18 = ipnt['pred_std_2018'].to_numpy()
        var_matrix_2018 = np.outer(std18, std18) * corr_matrix

        n = len(ipnt)
        agg_var_2009 = np.nansum(var_matrix_2009) / (n * n)
        agg_var_2018 = np.nansum(var_matrix_2018) / (n * n)
        
        # means
        m09 = ipnt['pred_2009'].mean()
        m18 = ipnt['pred_2018'].mean()

        # write back (index == id by design)
        plg.loc[plg.index == id_val, 'var_2009'] = agg_var_2009
        plg.loc[plg.index == id_val, 'var_2018'] = agg_var_2018
        plg.loc[plg.index == id_val, 'mean_2009'] = m09
        plg.loc[plg.index == id_val, 'mean_2018'] = m18
       
    # Merge back into plg
    plg['change'] = plg['mean_2018'] - plg['mean_2009']
    plg['signal'] = plg['change'].abs()
    plg['noise']  = np.sqrt(plg['var_2009'] + plg['var_2018'])

    plg.to_file(f'./material/grid_all.{isize}km_agg.calc.{version}.gpkg')


2.5 -------------------------------
10 -------------------------------
40 -------------------------------
200 -------------------------------


## calculate SNR, for visualization

In [20]:
for isize in sizes:
    ttprint(isize, '-------------------------------')
    plg = gpd.read_file(f'./material/grid_all.{isize}km_agg.calc.{version}.gpkg')
    plg['snr'] = plg['signal']/plg['noise']
    plg = plg.loc[plg['snr'].notna()]
    plg.to_file(f'./material/grid_all.{isize}km_agg.vlz.{version}.gpkg')

[08:34:19] 2.5 -------------------------------
[08:35:07] 10 -------------------------------
[08:35:10] 40 -------------------------------
[08:35:11] 200 -------------------------------


In [21]:
plg.to_file(f'./material/grid_all.200km_agg.vlz.20251017.gpkg')